In [118]:
import numpy as np
import requests
from bs4 import BeautifulSoup
import pandas as pd
import altair as alt

In [119]:
#read csv 
url = "https://www.ethnicity-facts-figures.service.gov.uk/health/diet-and-exercise/overweight-children/latest/downloads/overweight-children-2022-23.csv"
df = pd.read_csv(url)
df.head(20)

,measure,ethnicity,ethnicity_type,time,time_type,geography,geography_type,geography_code,gender,age,value,value_type,denominator,numerator,upper_95_c_i,lower_95_c_i
0,Percentage of children who are overweight (inc...,All,All,2014/15,Academic year,England,Country,E92000001,Persons,10-11 yrs,33.239710,%,531223.0,176577.0,33.36651,33.11316
1,Percentage of children who are overweight (inc...,All,All,2015/16,Academic year,England,Country,E92000001,Persons,10-11 yrs,34.166150,%,544615.0,186074.0,34.29222,34.04031
2,Percentage of children who are overweight (inc...,All,All,2016/17,Academic year,England,Country,E92000001,Persons,10-11 yrs,34.248060,%,556452.0,190574.0,34.37285,34.12348
3,Percentage of children who are overweight (inc...,All,All,2017/18,Academic year,England,Country,E92000001,Persons,10-11 yrs,34.321350,%,576574.0,197888.0,34.44401,34.19891
4,Percentage of children who are overweight (inc...,All,All,2018/19,Academic year,England,Country,E92000001,Persons,10-11 yrs,34.294840,%,600449.0,205923.0,34.41500,34.17487
5,Percentage of children who are overweight (inc...,All,All,2019/20,Academic year,England,Country,E92000001,Persons,10-11 yrs,35.189910,%,491138.0,172831.0,35.32358,35.05646
6,Percentage of children who are overweight (inc...,All,All,2020/21,Academic year,England,Country,E92000001,Persons,10-11 yrs,40.914490,%,124230.0,50828.0,41.18818,40.64137
7,Percentage of children who are overweight (inc...,All,All,2021/22,Academic year,England,Country,E92000001,Persons,10-11 yrs,37.759800,%,602000.0,227314.0,37.88234,37.63742
8,Percentage of children who are overweight (inc...,All,All,2022/23,Academic year,England,Country,E92000001,Persons,10-11 yrs,36.568320,%,613099.0,224200.0,36.68896,36.44785
9,Percentage of children who are overweight (inc...,All,All,2014/15,Academic year,England,Country,E92000001,Persons,4-5 yrs,21.885710,%,610636.0,133642.0,21.98959,21.78218


In [120]:
# read CC_data/~$summary-hbai-1994-95-2022-23-tables.ods sheet 1_4a
df_ods = pd.read_excel("summary-hbai-1994-95-2022-23-tables.ods", sheet_name="1_4a")

# make row 8 header
new_header = df_ods.iloc[6]
df_ods = df_ods[7:]
df_ods.columns = new_header
df_ods = df_ods.reset_index(drop=True)
df_ods = df_ods[8:]

# name the second column year
df_ods.columns.values[1] = 'year'
df_ods.columns.values[2] = 'Relative low income - before housing costs'
df_ods.columns.values[4] = 'Absolute low income'
df_ods.columns.values[7] = 'Relative low income - before housing costs'
df_ods.columns.values[9] = 'Relative low income'

# Correct the year format
df_ods['year'] = df_ods['year'].replace('2022/231', '2022/23')

# Drop the last row if it contains summary or non-relevant data
df_ods = df_ods[:-1]
df_ods = df_ods.reset_index(drop=True)
#df_ods 


In [121]:
# Convert year to string to avoid issues with Altair
df_ods['year'] = df_ods['year'].astype(str)

# Melt the dataframe to have a long format suitable for Altair
df_ods_melted = df_ods.melt(id_vars=['year'], value_vars=[
    'Absolute low income',
    'Relative low income'
], var_name='Income Type', value_name='Value')

# Make value numeric
df_ods_melted['Value'] = pd.to_numeric(df_ods_melted['Value'])

# Divide the value by 100
df_ods_melted['Value'] = df_ods_melted['Value'] / 100

# Create the Altair line chart
alt.Chart(df_ods_melted).mark_line().encode(
    x=alt.X('year', title=None),
    y=alt.Y('Value', axis=alt.Axis(format='%'), title=None, scale=alt.Scale(domain=[0, 0.5])),
    color='Income Type'
).properties(
    title={
        'text': 'Percentage of children living in poverty after housing costs',
        'subtitle': '% of children living below absolute/relative median income, 2002-2023. Source: FRS',
    },
).configure_title(
    anchor='start'
).configure_view(
    strokeWidth=0
).properties(
    width=600,
    height=400
)

#save json specs for vegalite
chart = alt.Chart(df_ods_melted).mark_line().encode(
    x=alt.X('year', title=None),
    y=alt.Y('Value', axis=alt.Axis(format='%'), title=None, scale=alt.Scale(domain=[0, 0.5])),
    color='Income Type'
).properties(
    title={
        'text': 'Percentage of children living in poverty after housing costs',
        'subtitle': '% of children living below absolute/relative median income, 2002-2023. Source: FRS',
    },
).configure_title(
    anchor='start'
).configure_view(
    strokeWidth=0
).properties(
    width=600,
    height=400
).to_json()

with open('CC3_fig1.json', 'w') as f:
    f.write(chart)

In [122]:
df = df[df["ethnicity"] == "All"]

In [123]:
#make altair line chart of value per age
alt.Chart(df).mark_line().encode(
    x='time',
    y='value',  
    color='age'
)

alt.Chart(...)

In [124]:
df['value'] = df['value'] / 100

#save json specs for vegalite
chart1 = alt.Chart(df).mark_line().encode(
    x=alt.X('time', title=None),
    y=alt.Y('value', axis=alt.Axis(format='%'), title=None, scale=alt.Scale(domain=[0, 0.5])),
    color=alt.Color('age', title=None)
).properties(
    title={
        'text': 'Percentage of children who are overweight',
        'subtitle': 'Estimates for reception year and year 6, 2014 - 2023. Source: NHS',
    },
).configure_title(
    anchor='start'
).configure_view(
    strokeWidth=0
).properties(
    width=600,
    height=400
).to_json()

with open('CC3_fig2.json', 'w') as f:
    f.write(chart1)